In [2]:
import pandas as pd
import numpy as np
import networkx as nx

### import all edges that nclude family and other

In [3]:
source_tar_df = pd.read_csv("D:/R_Project/Business_Group_project/raw/source_target_all_14010802.csv")

#### check id

In [4]:
source_tar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3778 entries, 0 to 3777
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Shareholder              3778 non-null   object 
 1   Percent                  3778 non-null   float64
 2   Compony                  3778 non-null   object 
 3   shareholder_id           3778 non-null   int64  
 4   company_national_id      3778 non-null   int64  
 5   shareholder_rasmio       3778 non-null   object 
 6   shareholder_national_id  3778 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 206.7+ KB


In [5]:
source_tar_df['shareholder_national_id'].apply(lambda x: len(str(x))).unique()

array([11], dtype=int64)

#### Functions for f_vector and malekiat matrix

In [6]:
def f_vector(source_tar_df,pyramid_head_id,):
    comp_df = source_tar_df[['Compony','company_national_id']]
    share_df = source_tar_df[['shareholder_rasmio','shareholder_national_id']]
    share_df.rename(columns={'shareholder_rasmio':'Compony','shareholder_national_id':'company_national_id'},inplace=True)
    nodes = pd.concat([comp_df,share_df])
    nodes.drop_duplicates(subset='company_national_id', inplace=True)
    nodes.reset_index(drop=True, inplace=True)
    pyramid_head_id = pyramid_head_id
    dir_share = source_tar_df[source_tar_df['shareholder_national_id']==pyramid_head_id][['shareholder_national_id','company_national_id','Percent']]
    py_dir_id = dir_share.company_national_id.to_list()
    py_dir_id.append(pyramid_head_id)
    ind_list = []
    ind_list.append(nodes[nodes['company_national_id']==pyramid_head_id].index[0])
    nodes_not_py=nodes.drop(ind_list).reset_index(drop=True)
    nodes_not_py['Percent']=0
    for i in dir_share['company_national_id']:
        a = nodes_not_py[nodes_not_py['company_national_id']==i].index[0]
        b = dir_share[dir_share['company_national_id']==i]['Percent'].index[0]
        nodes_not_py.iloc[a,2] = dir_share[dir_share['company_national_id']==i]['Percent'][b]
    nodes_not_py.rename(columns={'company_national_id':'Nationalcode','Compony':'Name','Percent':'Malekiat_Dolat_Dar_Sherkat'}, inplace=True)
    nodes_not_py.reset_index(drop=True,inplace=True)
    nodes_not_py = nodes_not_py[['Nationalcode','Name','Malekiat_Dolat_Dar_Sherkat']]
    #direct share of pyramid's head in business group firms
    nodes_not_py.to_csv('D:/R_Project/Business_Group_project/f_Vector_for_family.csv',encoding='utf-8-sig')
    nodes_not_py.to_excel('D:/R_Project/Business_Group_project/f_Vector_for_family.xlsx')
    return nodes_not_py

In [7]:
def matrix_malekiat(source_tar_df,pyramid_head_id):
    del_edge = []
    del_edge.extend(list(source_tar_df[source_tar_df['shareholder_national_id']==pyramid_head_id].index))
    del_edge.extend(list(source_tar_df[source_tar_df['company_national_id']==pyramid_head_id].index))
    source_tar_not_py=source_tar_df.drop(del_edge).reset_index(drop=True)
    dfn = f_vector(source_tar_df,pyramid_head_id)[['Nationalcode','Name']] 
    node = dfn.values.tolist()
    nodet=[]
    for i in node:
        i=tuple(i)
        nodet.append(i)
    n = list(dfn['Name'])
    nat = list(dfn['Nationalcode'])
    dfe = source_tar_not_py[['shareholder_national_id','company_national_id','Percent']]
    dfe.rename(columns={'shareholder_national_id':'source','company_national_id':'target','Percent':'weight'},inplace=True)
    e = dfe.values.tolist()
    et=[]
    for i in e:
        i = tuple(i)
        et.append(i)
    G=nx.DiGraph()
    G.add_nodes_from(nat)
    G.add_weighted_edges_from(et)
    M = nx.adjacency_matrix(G).todense()
    matrix = pd.DataFrame(M)
    matrix.to_excel("D:/R_Project/Business_Group_project/processed/malekiat_matrix_of_family.xlsx")
    matrix.to_csv("D:/R_Project/Business_Group_project/processed/malekiat_matrix_of_family.csv")
    return matrix

#### function executing

In [42]:
pyramid_head_id=10100334245

In [43]:
f_vector(source_tar_df,pyramid_head_id)

C:\Users\abyo0503\AppData\Local\Temp\ipykernel_6584\3190311721.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  share_df.rename(columns={'shareholder_rasmio':'Compony','shareholder_national_id':'company_national_id'},inplace=True)


,Nationalcode,Name,Malekiat_Dolat_Dar_Sherkat
0,10380244072,توريستي ورفاهي آبادگران ايران,0.0
1,10460039587,سيم و كابل ابهر,0.0
2,10100621228,لابراتوارداروسازي‌ دكترعبيدي‌,0.0
3,10102405220,سيمان آبيك,0.0
4,10260200698,كشت و صنعت آبشيرين,0.0
...,...,...,...
2012,14006704670,توسعه کسب و کار پارس سپنتا,0.0
2013,14004875152,پویش بازرگان ذوب آهن اصفهان,0.0
2014,10100947065,شهرکهای کشاورزی,0.0
2015,10100093996,سیلوهای ذخیره سازی و فرآوری غلات زر,0.0


In [44]:
matrix_malekiat(source_tar_df,pyramid_head_id)

C:\Users\abyo0503\AppData\Local\Temp\ipykernel_6584\3190311721.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  share_df.rename(columns={'shareholder_rasmio':'Compony','shareholder_national_id':'company_national_id'},inplace=True)
C:\Users\abyo0503\AppData\Local\Temp\ipykernel_6584\3082141898.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe.rename(columns={'shareholder_national_id':'source','company_national_id':'target','Percent':'weight'},inplace=True)
C:\Users\abyo0503\AppData\Local\Temp\ipykernel_6584\3082141898.py:24: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix i

,0,1,2,3,4,5,6,7,8,9,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
